In [1]:
%load_ext Cython

In [2]:
import numpy as np
import pandas as pd

path_dist = "C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/tenv/data/out/manhattan_nyc/lean_data/distance_matrix_km.npy"
dist_matrix = np.load(path_dist)
# dummy tour: 0, 1, 2, 3...
tour = np.array([i for i in range(500)])
print("There are", len(dist_matrix), "nodes")

There are 5378 nodes


In [3]:
print("Dummy tour:", tour)

Dummy tour: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 24

## Cython

In [4]:
%%cython
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport sqrt

cpdef check_cython(double[:,:] dist_matrix, int[:] tour_):
    cdef float dist
    cdef int i
    dist = 0
    for i in range(len(tour_)-1):
        dist+=dist_matrix[tour_[i]][tour_[i+1]]
    
    return dist

cpdef two_opt_cython(double[:,:] dist_matrix, int[:] tour_):
    cdef float min_change, change
    cdef int i, j, min_i, min_j, num_cities
    num_cities = len(tour_)
    min_change = 0
    # Find the best move
    for i in range(len(tour_) - 2):
        for j in range(i + 2, len(tour_) - 1):
            change = dist_matrix[tour_[i]][tour_[j]] + dist_matrix[tour_[i+1]][tour_[j+1]] - dist_matrix[tour_[i]][tour_[i+1]] - dist_matrix[tour_[j]][tour_[j+1]]
            if change < min_change:
                min_change = change
                min_i, min_j = i, j

    # Update tour with best move
    if min_change < 0:
        tour_[min_i+1:min_j+1] = tour_[min_i+1:min_j+1][::-1]
    return np.asarray(tour_)  # memoryview to numpy array

In [5]:
%time two_opt_cython(dist_matrix, tour.astype('int32'))

Wall time: 1.99 ms


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166,
       165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153,
       152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140,
       139, 138, 137, 136, 135, 134, 133, 176, 177, 178, 179, 18

In [6]:
best = two_opt_cython(dist_matrix, tour.astype('int32'))

In [7]:
best = two_opt_cython(dist_matrix, tour.astype('int32'))
print("tour:", check_cython(dist_matrix, tour), "--- 2-Opt:", check_cython(dist_matrix, best))

tour: 928.0270385742188 --- 2-Opt: 897.118896484375


## Python

In [8]:
def two_opt_python(tour_):
    min_change = 0

    # Find the best move
    for i in range(len(tour_) - 2):
        for j in range(i + 2, len(tour_) - 1):
            change = dist_matrix[tour_[i]][tour_[j]] + dist_matrix[tour_[i+1]][tour_[j+1]] - dist_matrix[tour_[i]][tour_[i+1]] - dist_matrix[tour_[j]][tour_[j+1]]
            if change < min_change:
                min_change = change
                min_i, min_j = i, j
    # Update tour with best move
    if min_change < 0:
        tour[min_i+1:min_j+1] = tour[min_i+1:min_j+1][::-1]    
    
    return np.asarray(tour_)

def check_python(tour_):
    dist = 0
    for i in range(len(tour_)-1):
        dist+=dist_matrix[tour_[i]][tour_[i+1]]
    return dist


In [9]:
%time two_opt_python(tour.astype('int32'))

Wall time: 561 ms


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [10]:
%time check_python(tour)

Wall time: 0 ns


897.1184616128817

In [11]:
best = two_opt_python(tour.astype('int32'))
print("tour:", check_python(tour), "--- 2-Opt:", check_python(best))

tour: 865.7404377922163 --- 2-Opt: 897.1184616128817


In [12]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force
#cython: boundscheck=False, wraparound=False, nonecheck=False
import numpy as np
from math import exp 
from libc.math cimport exp as c_exp
from cython.parallel import prange

cpdef check_cython(double[:,:] dist_matrix, int[:] tour_):
    cdef float dist
    cdef int i
    dist = 0
    for i in range(len(tour_)-1):
        dist+=dist_matrix[tour_[i]][tour_[i+1]]
    
    return dist

def c_array_f_multi(float[:] X):

    cdef int N = X.shape[0]
    cdef double[:] Y = np.zeros(N)
    cdef int i

    for i in prange(N, nogil=True):
        if X[i] > 0.5:
            Y[i] = c_exp(X[i])
        else:
            Y[i] = 0

    return np.asarray(Y)

In [15]:
a = np.linspace(0, 10, 1000).astype('float32')

In [18]:
%timeit c_array_f_multi(a)

13.6 µs ± 796 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [23]:
import math

In [26]:
%timeit np.exp(a)

6.79 µs ± 753 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
